In [ ]:
# ONLY FOR ONE TIME RUN
# 2 ways of installing: Through anaconda navigator->environments etc & through terminal of a specific environment *it also opens at
# anaconda navigator->environments->'play button'

# for tweepy we need to add 'conda-forge' channel in the anaconda environment and update (channels). Then, we can find tweepy in the anaconda list of packages
# for mysql.connector I add 'mysql-connector-python'
# for matplotlib I can use the alternative way of intalling: I write 'pip install matplotlib' in conda prompt
# for textblob I write in prompt 'pip install textblob'
# for pandas I write in prompt 'pip install pandas'

In [1]:
import sys  # ??
import re
import tweepy
import mysql.connector
from mysql.connector import Error   #Gia na fortwsw ta eidika errors gia sql connection
import matplotlib.pyplot as plt
from textblob import TextBlob
import pandas as pd
import numpy as np
from datetime import datetime
import string  # ??
import nltk  # ??
from nltk.stem import WordNetLemmatizer  # ??

# Tweets Fetching
Needed steps:
- Create a class inheriting from StreamListener
- Using that class create a Stream object
- Connect to the Twitter API using the Stream.

Notes: <br>
- The Key Differences Between Streaming APIs and REST APIs: https://www.nstream.io/the-key-differences-between-streaming-apis-and-rest-apis/ <br>
UNFORTUNATELY, Streaming tweets are not permitted in free and basic versions of Twitter Developer
https://stackoverflow.com/questions/76445414/twitter-streams-api-returns-403

- Tweepy docmentation:
https://docs.tweepy.org/en/stable/index.html

We will use Twitter API v2, which uses 'Client' method (or interface) that returns a Twitter API object<br>
https://docs.tweepy.org/en/stable/client.html#tweepy.Client
https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9

Twitter Developer Portal
https://developer.twitter.com/en/portal/dashboard

Twitter queries
https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query


In [4]:
# AUTHENTICATION of Twitter Client

# # Configuration of the twitter account
p_consumer_key = 'SbPzmphYe7DHRMB0QPnYCuj1D'
p_consumer_secret = 'Wod9VZS49DHxKhXKc5TL6yO2yrzZpeakvUUH1bufrspbBt8ygh'
p_access_token = '1485552804655685639-zjmpYuEc1AaZ1dQuLTdKaMyq72SQfW'
p_access_token_secret = '829gYuJgv79BprhxxP1TFCvkJigL7mtFqtqFIwcvYu6xv'
p_bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEd1YgEAAAAAOH7PeGJv9rqSuF%2F2It5VB%2BX3S2g%3DPsP4UHNi0bwZEYoUmz5EwyAPRVny9bl9iChLNPFy0MvAw5Een2'

In [7]:
# USE CLIENT class to search for tweets (not real time tweets)

p_client = tweepy.Client(bearer_token=p_bearer_token, consumer_key=p_consumer_key, consumer_secret=p_consumer_secret, access_token=p_access_token, access_token_secret=p_access_token_secret)
p_query = 'python'
# p_tweet_fields = ['context_annotations', 'created_at','text','lang','geo','id']
p_tweet_fields = ['text']
p_user_fields = ['name', 'id', 'location','username']

# tweets = p_client.search_recent_tweets(query=p_query, tweet_fields=p_tweet_fields, user_fields = p_user_fields, max_results=10)
tweets = p_client.search_recent_tweets(query=p_query, tweet_fields=p_tweet_fields)

for tweet in tweets.data:
    print(user.name)



# #each of those lists will contain the values for a column in the dataframe
# list1=[]
# list2=[]
# list3=[]
# list4=[]
# list5=[]
# list6=[]
# list7=[]
# list8=[]
# list9=[]

# # Pass the tweets' metadata in lists
# for tweet in tweets:
#     list1.append(tweet.text)
#     list2.append(tweet.user.screen_name)
#     list3.append(tweet.created_at)
#     list4.append(tweet.retweet_count)
#     list5.append(tweet.user.location)
#     list6.append(tweet.user.followers_count)
#     list7.append(tweet.user.statuses_count)
#     list8.append(tweet.entities['hashtags'])

# # Create the dataframe
# data = pd.DataFrame(list2, columns=["user"])
# data.insert(loc=1, column='tweet', value=list1)
# data.insert(loc=2, column='date', value=list3)
# data.insert(loc=3, column='n_ret', value=list4)
# data.insert(loc=4, column='loc', value=list5)
# data.insert(loc=5, column='n_fol', value=list6)
# data.insert(loc=6, column='n_twe', value=list7)
# data.insert(loc=7, column='hashtags', value=list8)

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

# CONNECT TO DATABASE TO STORE RAW DATA

In [ ]:
db_name = 'case_st_twit'

# CONNECT TO DATABASE AND CREATE THE TABLE THAT WE WILL STORE THE TWEETS
try:
    db = mysql.connector.connect(host='localhost', database = db_name , user='root', password='')
    if db.is_connected():
        print('Connected to MySQL Database')
        cur = db.cursor()  # create object cursor from cursor method

        # Creating the table of database
        q = "CREATE table r_tweets( \
                username VARCHAR(30) PRIMARY KEY, \
                tweet VARCHAR(280), \
                t_date VARCHAR(30), \
                n_ret INT(3), \
                loc VARCHAR(20), \
                n_fol INT(9), \
                n_twe INT(3), \
                hashtags VARCHAR(50), \
                commit;"
        cur.execute(q)
except Error as e:
    print(e)

finally:
    cur.execute("commit;")
    db.close()
    print('Database connection closed!')

# CONNECT TO DATABASE TO STORE THE FETCHED TWEETS AS RAW DATA
try:
    db = mysql.connector.connect(host='localhost', database= db_name, user='root', password='')
    if db.is_connected():
        cur = db.cursor()

        for row in range(len(list1)):
            # Before running the sql commands, we clean the text data from apostrophes <'>, in order to avoid losing data due to faulty comments in sql commands
            q = "INSERT INTO tweets (username, tweet, t_date, n_ret, loc, n_fol, n_twe, hashtags) VALUES ('" \
            + re.sub(r'[\']', ' ',str(list2[row])) + "', '" + re.sub(r'[\']', ' ',str(list1[row])) + "', '" + str(list3[row]) + \
            "', '" + str(list4[row]) + "', '" + re.sub(r'[\']', ' ',str(list5[row])) + "', '" + str(list6[row]) + \
            "', '" + str(list7[row]) + "', '"  + re.sub(r'[\']', ' ',str(list8[row])) + \
            "');"

            try:
                print(q)
                cur.execute(q)
            except Error as e:
                print(e)

except Error as e:
    print(e)

finally:
    cur.execute("commit;")
    db.close()


# CLEANING OF THE TWEETS

In [ ]:
# Load the raw data from database
try:
    db = mysql.connector.connect(host='localhost', database = db_name, user='root', password='')

    # Initialization for storing the tweets in pandas object 'data'
    data = []
    columns = ['user', 'tweet', 'date', 'n_ret', 'loc', 'n_fol', 'n_twe', 'hashtags']
    if db.is_connected():
        cur = db.cursor()  # create object cursor from cursor method

        q = "SELECT * FROM tweets;"
        cur.execute(q)
        #Create the dataframe with all the data from database
        data = pd.DataFrame(cur.fetchall(), columns=columns)

except Error as e:
    print(e)

finally:
    cur.execute("commit;")
    db.close()


# Remove duplicate users. We want one tweet from every user
data.drop_duplicates(subset ="user", keep = False, inplace = True)

positive = 0
negative = 0
neutral = 0
polarity = 0
list9=[]
for row in data.index:
    # Remove punctuations
    cleaned_tweet = ''.join(re.sub ("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])| (\w+:\ / \ / \S+)", " ", data['tweet'][row]))
    # Remove the words with the @ symbol
    cleaned_tweet = re.sub ('@\S+', '', cleaned_tweet)
    # Remove the Url's
    cleaned_tweet = re.sub('((www.[^s]+)|(https?://[^s]+))', ' ', cleaned_tweet)
    # Remove the hashtags
    cleaned_tweet = re.sub ('#\S+', '', cleaned_tweet)
    # Remove the change row
    cleaned_tweet = re.sub ('\\n', '', cleaned_tweet)
    # Remove Stopwords
    stopwords = set(["", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "A", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "after", "afterwards", "ag", "again", "against", "ah", "ain", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appreciate", "approximately", "ar", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "B", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "been", "before", "beforehand", "beginnings", "behind", "below", "beside", "besides", "best", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "C", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "ci", "cit", "cj", "cl", "clearly", "cm", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "could", "couldn", "couldnt", "course", "cp", "cq", "cr", "cry", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d", "D", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "dj", "dk", "dl", "do", "does", "doesn", "doing", "don", "done", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "E", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "F", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "G", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "H", "h2", "h3", "had", "hadn", "happens", "hardly", "has", "hasn", "hasnt", "have", "haven", "having", "he", "hed", "hello", "help", "hence", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hh", "hi", "hid", "hither", "hj", "ho", "hopefully", "how", "howbeit", "however", "hr", "hs", "http", "hu", "hundred", "hy", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "im", "immediately", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "inward", "io", "ip", "iq", "ir", "is", "isn", "it", "itd", "its", "iv", "ix", "iy", "iz", "j", "J", "jj", "jr", "js", "jt", "ju", "just", "k", "K", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "ko", "l", "L", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "M", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "my", "n", "N", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "neither", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "O", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "otherwise", "ou", "ought", "our", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "P", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "pp", "pq", "pr", "predominantly", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "Q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "R", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "S", "s2", "sa", "said", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "seem", "seemed", "seeming", "seems", "seen", "sent", "seven", "several", "sf", "shall", "shan", "shed", "shes", "show", "showed", "shown", "showns", "shows", "si", "side", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somehow", "somethan", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "sz", "t", "T", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "thereof", "therere", "theres", "thereto", "thereupon", "these", "they", "theyd", "theyre", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "U", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "used", "useful", "usefully", "usefulness", "using", "usually", "ut", "v", "V", "va", "various", "vd", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "W", "wa", "was", "wasn", "wasnt", "way", "we", "wed", "welcome", "well", "well-b", "went", "were", "weren", "werent", "what", "whatever", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "whom", "whomever", "whos", "whose", "why", "wi", "widely", "with", "within", "without", "wo", "won", "wonder", "wont", "would", "wouldn", "wouldnt", "www", "x", "X", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "Y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "your", "youre", "yours", "yr", "ys", "yt", "z", "Z", "zero", "zi", "zz"])
    cleaned_tweet = ' '.join([word for word in str(cleaned_tweet).split() if word not in stopwords])

    # Calculate the polarity of the sentiment analysis using textblob and store the results in the sentiment column of the dataframe
    analysis = TextBlob(cleaned_tweet)
    list9.append(analysis.sentiment.polarity)

    if (analysis.sentiment.polarity == 0):
        neutral += 1
    elif (analysis.sentiment.polarity < 0):
        negative += 1
    elif (analysis.sentiment.polarity > 0):
        positive += 1

data.insert(loc=8, column='polarity', value = list9)



# STORE THE CLEANED TWEETS IN DATABASE 

In [ ]:
# CREATE A NEW TABLE IN DATABASE cl_tweets
try:
    db = mysql.connector.connect(host='localhost', database='twit_sent_an', user='root', password='')
    if db.is_connected():
        cur = db.cursor()  # create object cursor from cursor method

        q = "CREATE table cl_tweets( \
                username VARCHAR(30) PRIMARY KEY, \
                tweet VARCHAR(280), \
                t_date VARCHAR(30), \
                n_ret INT(3), \
                loc VARCHAR(20), \
                n_fol INT(9), \
                n_twe INT(3), \
                hashtags VARCHAR(50), \
                sentiment FLOAT(4,3)); \
                commit;"
        cur.execute(q)
except Error as e:
    print(e)

finally:
    cur.execute("commit;")
    db.close()


# STORE THE CLEANED TWEETS WITH THE SENTIMENT ANALYSIS RESULTS IN A NEW TABLE IN DATABASE cl_tweets
try:
    db = mysql.connector.connect(host='localhost', database='twit_sent_an', user='root', password='')
    if db.is_connected():
        print('Connected to MySQL Database')
        cur = db.cursor()  # create object cursor from cursor method

        for row in data.index:
            q = "INSERT INTO cl_tweets (username, tweet, t_date, n_ret, loc, n_fol, n_twe, hashtags, sentiment) VALUES ('" \
            + str(data['user'][row]) + "', '" + str(data['tweet'][row]) + "', '" + str(data['date'][row]) + \
            "', '" + str(data['n_ret'][row]) + "', '" + str(data['loc'][row]) + "', '" + str(data['n_fol'][row]) + \
            "', '" + str(data['n_twe'][row]) + "', '"  + str(data['hashtags'][row]) + \
            "', '" + str(data['polarity'][row]) + "');"

            cur.execute(q)

except Error as e:
    print(e)

finally:
    cur.execute("commit;")
    db.close()
    print('Database connection closed!')

In [ ]:

mega_positive=0
mega_negative=0
mega_neutral=0
mini_positive=0
mini_negative=0
mini_neutral=0

list10=[]
#list 10 is going to accept 2 values (1 for the people that have more followers than average and 0
#for those who have followers less than average)
mean_followers = sum(data['n_fol'])/ len(data['n_fol'])
for element in data['n_fol']:
    if element > mean_followers:
        list10.append(1)
    else:
        list10.append(0)

data.insert(loc=9, column='weight', value=list10)

# 9th and 10th column in the dataframe contain the polarity value and the corresponding weight whether
#a user has followers above average or not
#So mega_positive will get +1 when we meet a user that has followers above average and polarity > 0
#Thus, mini positive will get +1 when we meet a user that has followers below average and polarity > 0 etc.
for index,row in data.iterrows():
    if row['polarity'] > 0 and row['weight'] == 1:
        mega_positive += 1
    elif row['polarity'] > 0 and row['weight'] == 0:
        mini_positive += 1
    elif row['polarity'] < 0 and row['weight'] == 1:
        mega_negative += 1
    elif row['polarity'] < 0 and row['weight'] == 0:
        mini_negative += 1
    elif row['polarity'] == 0 and row['weight'] == 1:
        mega_neutral += 1
    else:
        mini_neutral += 1


#we want to have the percentage each opinion covers in the pie we are going to create
positive =(positive/ noOfSearchTerms)*100
negative =(negative/ noOfSearchTerms)*100
neutral =(neutral/ noOfSearchTerms)*100

final_positive = mega_positive*0.7 + mini_positive*0.3
final_negative = mega_negative*0.7 + mini_negative*0.3
final_neutral = mega_neutral*0.7 + mini_neutral*0.3


print("how people are reacting on " + searchTerm + "by analyzing" + str(noOfSearchTerms) + "Tweets.")

if (polarity == 0):
    print("Neutral")
elif (polarity < 0):
    print("Negative")
else:
    print("Positive")

#plotting the polarity of objects
labels = ['Positive [' +str(positive)+'%]', 'Neutral [' +str(neutral)+ '%]', 'Negative [' +str(negative) + '%]']

sizes = [positive, neutral, negative]
colors = ['green', 'yellow', 'red']
patches, text = plt.pie(sizes, colors=colors)

plt.legend(patches, labels, loc='best')
plt.title('what people think about ' +searchTerm+'by analyzing '+str(noOfSearchTerms)+'Tweets.')
plt.axis('equal')
plt.tight_layout()
plt.show()

#plotting the impact that people with more followers have on the objects
labels1 = ['Positive ' +str(final_positive)+' impact', 'Neutral ' +str(final_neutral)+ ' impact', 'Negative ' +str(final_negative) + ' impact']

sizes1 = [final_positive, final_neutral, final_negative]
colors1= ['green', 'yellow', 'red']
patches1, text1 = plt.pie(sizes, colors=colors)

plt.legend(patches1, labels1, loc='best')
plt.title('what people think about ' +searchTerm+'by analyzing '+str(noOfSearchTerms)+'Tweets.')
plt.axis('equal')
plt.tight_layout()
plt.show()

# Dataset separation based on the location. Here we use two csv's tha contain some cities in USA and Europe
abbUS = pd.read_csv('AbbrevUSA.csv')
abbEU = pd.read_csv('AbbrevEU.txt')
usa = []
EU = []
other = []
for index,row in data.iterrows():
    if row['loc'] in abbUS.values:
        usa.append(index)
    elif row['loc'] in abbEU.values:
        EU.append(index)
    elif row['loc']!= '':
        other.append(index)

# Dataset separation based on the platform
net = []
disney = []
amazon = []
net_sum = 0
dis_sum = 0
am_sum = 0
for index,row in data.iterrows():
    if 'netflix' in row['tweet'].lower():
        net.append(index)
        net_sum += row['polarity']
    elif 'disney' in row['tweet'].lower():
        disney.append(index)
        dis_sum += row['polarity']
    elif 'amazon' in row['tweet'].lower():
        amazon.append(index)
        am_sum += row['polarity']

# Preperation of the data in order to plot the polarity of the tweets based on Location info
usa_net_sum = 0
usa_dis_sum = 0
usa_am_sum = 0
eu_net_sum = 0
eu_dis_sum = 0
eu_am_sum = 0
oth_net_sum = 0
oth_dis_sum = 0
oth_am_sum = 0
usa_net_len = 0
usa_dis_len = 0
usa_am_len = 0
eu_net_len = 0
eu_dis_len = 0
eu_am_len = 0
oth_net_len = 0
oth_dis_len = 0
oth_am_len = 0
for index,row in data.iterrows():
    if index in usa:
        if index in net:
            usa_net_sum += row['polarity']
            usa_net_len += 1
        if index in disney:
            usa_dis_sum += row['polarity']
            usa_dis_len += 1
        if index in amazon:
            usa_am_sum += row['polarity']
            usa_am_len += 1
    elif index in EU:
        if index in net:
            eu_net_sum += row['polarity']
            eu_net_len +=1
        if index in disney:
            eu_dis_sum += row['polarity']
            eu_dis_len += 1
        if index in amazon:
            eu_am_sum += row['polarity']
            eu_am_len += 1
    elif index in other:
        if index in net:
            oth_net_sum += row['polarity']
            oth_net_len += 1
        if index in disney:
            oth_dis_sum += row['polarity']
            oth_dis_len += 1
        if index in amazon:
            oth_am_sum += row['polarity']
            oth_am_len += 1

Loc_graph = [[usa_net_sum/usa_net_len,eu_net_sum/eu_net_len,oth_net_sum/oth_net_len],
             [usa_dis_sum/usa_dis_len,eu_dis_sum/eu_dis_len,oth_dis_sum/oth_dis_len],
             [usa_am_sum/usa_am_len,eu_am_sum/eu_am_len,oth_am_sum/oth_am_len]]

# Plot the polarity of the tweets based on Location info
X = np.arange(3)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Average polarity')
ax.set_title('Average opinion for each platform in USA, EU and rest of the world')
ax.set_xticks([0.3,1.3,2.3], ('Netflix', 'Disney Plus', 'Amazon Prime'))
ax.bar(X + 0.00, Loc_graph[0], color = 'b', width = 0.25, label='USA')
ax.bar(X + 0.25, Loc_graph[1], color = 'g', width = 0.25, label='EU')
ax.bar(X + 0.50, Loc_graph[2], color = 'r', width = 0.25, label='other')
plt.legend()
plt.grid()

# Preperation of the data in order to plot the polarity of the tweets based on time info
time = []
for row in data.index:
    time.append(datetime.strptime(data['date'][row][0:19],'%Y-%m-%d %H:%M:%S'))

t8_net_sum = 0
t8_dis_sum = 0
t8_am_sum = 0
t14_net_sum = 0
t14_dis_sum = 0
t14_am_sum = 0
t24_net_sum = 0
t24_dis_sum = 0
t24_am_sum = 0
t8_net_len = 0
t8_dis_len = 0
t8_am_len = 0
t14_net_len = 0
t14_dis_len = 0
t14_am_len = 0
t24_net_len = 0
t24_dis_len = 0
t24_am_len = 0
for index,row in data.iterrows():
    if time[index] < datetime.strptime('2022-02-25 19:00:00','%Y-%m-%d %H:%M:%S'):
        if index in net:
            t8_net_sum += row['polarity']
            t8_net_len += 1
        if index in disney:
            t8_dis_sum += row['polarity']
            t8_dis_len += 1
        if index in amazon:
            t8_am_sum += row['polarity']
            t8_am_len += 1
    elif time[index] < datetime.strptime('2022-02-25 20:00:00','%Y-%m-%d %H:%M:%S'):
        if index in net:
            t14_net_sum += row['polarity']
            t14_net_len += 1
        if index in disney:
            t14_dis_sum += row['polarity']
            t14_dis_len += 1
        if index in amazon:
            t14_am_sum += row['polarity']
            t14_am_len += 1
    else:
        if index in net:
            t24_net_sum += row['polarity']
            t24_net_len += 1
        if index in disney:
            t24_dis_sum += row['polarity']
            t24_dis_len += 1
        if index in amazon:
            t24_am_sum += row['polarity']
            t24_am_len += 1

Time_graph = [[t8_net_sum/t8_net_len,t14_net_sum/t14_net_len,t24_net_sum/t24_net_len],
             [t8_dis_sum/t8_dis_len,t14_dis_sum/t14_dis_len,t24_dis_sum/t24_dis_len],
             [t8_am_sum/t8_am_len,t14_am_sum/t14_am_len,t24_am_sum/t24_am_len]]

# Plot the polarity of the tweets based on time info
X = np.arange(3)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Average polarity')
ax.set_title('Average opinion for each platform in different time periods')
ax.set_xticks([0.3,1.3,2.3], ('before 19:00', 'before 20:00', 'after 20:00'))
ax.bar(X + 0.00, Time_graph[0], color = 'b', width = 0.25, label='Netflix')
ax.bar(X + 0.25, Time_graph[1], color = 'g', width = 0.25, label='Disney Plus')
ax.bar(X + 0.50, Time_graph[2], color = 'r', width = 0.25, label='Amazon Prime')
plt.legend()
plt.grid()

# PROXEIRO

In [ ]:
# GIA TUXAIA EPILOGH YPARXONTWN TWEETS

searchTerm = '.....'
noOfSearchTerms = 1000

tweets = tweepy.Cursor(api.search_tweets, q=searchTerm, lang='en').items(noOfSearchTerms)
#each of those lists will contain the values for a column in the dataframe
list1=[]
list2=[]
list3=[]
list4=[]
list5=[]
list6=[]
list7=[]
list8=[]
list9=[]

# Pass the tweets' metadata in lists
for tweet in tweets:
    list1.append(tweet.text)
    list2.append(tweet.user.screen_name)
    list3.append(tweet.created_at)
    list4.append(tweet.retweet_count)
    list5.append(tweet.user.location)
    list6.append(tweet.user.followers_count)
    list7.append(tweet.user.statuses_count)
    list8.append(tweet.entities['hashtags'])

# Create the dataframe
data = pd.DataFrame(list2, columns=["user"])
data.insert(loc=1, column='tweet', value=list1)
data.insert(loc=2, column='date', value=list3)
data.insert(loc=3, column='n_ret', value=list4)
data.insert(loc=4, column='loc', value=list5)
data.insert(loc=5, column='n_fol', value=list6)
data.insert(loc=6, column='n_twe', value=list7)
data.insert(loc=7, column='hashtags', value=list8)

In [ ]:
# Mporei na mpei STO ONLINE TWEETS FETCHING

# Enallaktika apothikeush twn tweets kateutheian sthn database
        try:
            db = mysql.connector.connect(host='localhost', database='twit_sent_an', user='root', password='')
            if db.is_connected():
                #print('Connected to MySQL Database')
                cur = db.cursor()  # create object cursor from cursor method

                q = "INSERT INTO tweets (username, n_fol, n_twe, n_ret, Tweet, T_date, Loc, Hashtags, sentiment) VALUES ('" \
                + re.sub(r'[\']', ' ', str(status.user.screen_name)) + "', '" + str(status.user.followers_count) + "', '" + str(status.user.statuses_count) + \
                "', '" + str(status.retweet_count) + "', '" + re.sub(r'[\']', ' ',str(status.text)) + "', '" + str(status.created_at) + \
                "', '" + str(status.user.location) + "', '" + re.sub(r'[\']', ' ',str(htags)) + "', '" + str(None) + \
                "');"

                cur.execute(q)

        except Error as e:
            print(e)

        finally:
            cur.execute("commit;")
            # cur.execute("SELECT * FROM tweets;")
            # data = cur.fetchall()
            # df = pd.DataFrame(data, columns = columns)
            db.close()
            #print('Database connection closed!')

In [ ]:
# STREAMING IS NOT PERMITTED ON FREE VERSIONS ANY MORE !!!!

# Creation of Listener class, which contains the methods we 
# want with the settings we want from the methods of 
# Streaming Client object

# Na dw ti akrivws kanei h parakatw grammh
# Dhmiourgei class me antigrafh? kanei edit mia class?
# Epishs to exw grapsei swsta? 
class MyListener(tweepy.StreamingClient):  # Inherit class'tweepy.StreamingClient' to my new class'MyListener'
    
    def on_connect(self):
        print("Connected!")
    
    # Initialization for storing the tweets in object (list) 'data'
    ##@@ Edw mporw na xrhsimopoihsw dataframe anti gia list???  @@@
    
    data = []
    columns = ['User', 'n_fol', 'n_twe', 'n_ret', 'Tweet', 'Date', 'Loc', 'Hashtags', 'Sentiment']

    def on_tweet(self, tweet):  #receives a tweet and processes it according to the conditions, if there are any, and adds the tweet to the hashmap
        print(f"{tweet.id} {tweet.created_at} ({tweet.author_id}): {tweet.text}")
        print("-"*50)
        
        #enallaktika
        if tweet.referenced_tweets == None:
            # self.new_tweet[“tweet”] = tweet.text
            print(tweet.text)
            time.sleep(0.3)
        
        # Perasma twn tweets se list
        user = self.get_user(username=screen_name)
        print(user)
#         self.data.append([status.user.screen_name, status.user.followers_count, status.user.statuses_count,
#                          status.retweet_count, status.text, str(status.created_at), status.user.location,
#                          htags, None])
    
    def on_includes(self, includes):  #is responsible for the user details and adds the user data to the hashmap
        self.new_tweet["username"] = includes["users"][0].username
        print(self.new_tweet)
#         # insert tweets in db
#         cursor.execute(
#         "INSERT INTO tweets VALUES (?,?)",
#         (
#         self.new_tweet["username"],
#         self.new_tweet["tweet"],
#         ),
#         )
#         conn.commit()
#         # print(self.new_tweet)
#         print("tweet added to db!")
#         print("-" * 30)
   
    
    # prints the error
    def on_error(self, status):
        print(status)

####   APORIES
    # Travame to created_at san string   [datetime.datetime(2022, 2, 20, 11, 5, 25, tzinfo=datetime.timezone.utc)]
    # Travame mono to text apo ta hashtags [{'text': 'Unbelievable', 'indices': [8, 21]}]

In [ ]:
# ONLINE TWEETS FETCHING
new_Tweet = MyListener(bearer_token)
# new_Tweet = tweepy.StreamingClient(bearer_token)

new_Tweet.sample()

# add new rules    
# rule = tweepy.StreamRule(value="Python")
# new_Tweet.add_rules(rule)
 
# new_Tweet.filter()
# new_Tweet.filter(expansions="author_id",tweet_fields="created_at")


# add the keywords for the streamer. It returns JSON formatted dictionary object
# new_Tweet.filter(languages=['en', 'gr'], track=['fires', 'Greece'])